In [ ]:
# old code to place order according buy first and according to the market change the signals 

# Main trading logic
order_count = 0
pnl = 1  # Start with profit state
inPosition = False
buying_price = 0
selling_price = 0

# pnl = find_pos()

while order_count < 100:
    current_time = datetime.datetime.now().time()
    if current_time >= datetime.time(9, 0) and current_time <= datetime.time(15,0):
        if not inPosition:
            # the strategy is starts from here
            if pnl == 1:
                buy_order = place_order('B')
                res = client.order_report()
                ltp = float(redis_client.hget('stock_data', TOKEN))
                buying_price = float(res['data'][-1]['avgPrc'])
                
                if buy_order and buy_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Bought at {buying_price} , Ltp is : {ltp}")
                    
                    inPosition = True
                    tar = buying_price * 1.001
                    sl = buying_price * 0.996
                else:
                    print(f"Error in buying the order {buy_order}")
            else:
                sell_order = place_order('S')
                
                ltp = float(redis_client.hget('stock_data', TOKEN))
                res = client.order_report()
                selling_price = float(res['data'][-1]['avgPrc'])
                if sell_order and sell_order.get('stat') == 'Ok':
                    order_count += 1
                    print(f"Sold at {selling_price}")
                    # order_count += 1
                    inPosition = True
                    tar = selling_price * 0.999
                    sl = selling_price * 1.003
                else:
                    print(f"Error in selling the order {sell_order} , Ltp is : {ltp}")
        
        if inPosition:
            ltp = float(redis_client.hget('stock_data', TOKEN))
            if pnl == 1:  # Currently holding a buy position
                if ltp >= tar:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Profit!")
                        pnl = 1
                        order_count += 1
                        inPosition = False
                        # time.sleep(5)
                elif ltp <= sl:
                    sell_order = place_order('S')
                    
                    if sell_order and sell_order.get('stat') == 'Ok':
                        print(f"Sold at {ltp}, Loss!")
                        pnl = 0
                        order_count += 1
                        inPosition = False
            else:  # Currently holding a sell position
                if ltp <= tar:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Profit!")
                        pnl = 0
                        inPosition = False
                        # time.sleep(5)
                elif ltp >= sl:
                    buy_order = place_order('B')
                    order_count += 1
                    if buy_order and buy_order.get('stat') == 'Ok':
                        print(f"Bought at {ltp}, Loss!")
                        pnl = 1
                        inPosition = False

            time.sleep(0.25)  # Check every 1 second
    else:
        print("Market is closed.")
        break

print("Order reached order_count.")